In [1]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [48]:
??LogisticRegression

In [31]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
subm = pd.read_csv('sample_submission.csv')

In [4]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['none'] = 1-train[label_cols].max(axis=1)
train.describe()


,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805,0.898321
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420,0.302226
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
print(len(train),len(test))
COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)
len(train),len(test)

159571 153164


(159571, 153164)

In [10]:
import re, string

In [21]:
re_tok = re.compile(r'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [22]:
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train[COMMENT])
test_term_doc = vec.transform(test[COMMENT])

In [23]:
trn_term_doc, test_term_doc

(<159571x74191 sparse matrix of type '<class 'numpy.float64'>'
 	with 26086676 stored elements in Compressed Sparse Row format>,
 <153164x74191 sparse matrix of type '<class 'numpy.float64'>'
 	with 22596069 stored elements in Compressed Sparse Row format>)

In [24]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [25]:
x = trn_term_doc
test_x = test_term_doc

In [26]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [27]:
preds = np.zeros((len(test), len(label_cols)))

for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

fit toxic
fit severe_toxic
fit obscene
fit threat
fit insult
fit identity_hate


In [32]:
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
submission.to_csv('submission.csv', index=False)

In [46]:
print(x[:,2])
print(x.shape)

  (145, 0)	0.0840832561758
  (416, 0)	0.119980059064
  (460, 0)	0.0482918050875
  (469, 0)	0.21709536447
  (513, 0)	0.198190766765
  (526, 0)	0.149426551821
  (570, 0)	0.0937387808779
  (768, 0)	0.0990600416162
  (977, 0)	0.0651769703439
  (1433, 0)	0.362456303315
  (1457, 0)	0.116251421943
  (1556, 0)	0.175305029034
  (1760, 0)	0.127968585269
  (1798, 0)	0.159858765254
  (2048, 0)	0.238287044174
  (2136, 0)	0.0526404629857
  (2290, 0)	0.119506622271
  (2379, 0)	0.151144684526
  (2606, 0)	0.0707783296507
  (2928, 0)	0.125550649113
  (3058, 0)	0.124076913403
  (3114, 0)	0.17261241236
  (3148, 0)	0.0882614907989
  (3593, 0)	0.121426149663
  (3672, 0)	0.05649470783
  :	:
  (154487, 0)	0.266401034088
  (154819, 0)	0.0493701600066
  (154914, 0)	0.118028340034
  (155158, 0)	0.101599755498
  (155304, 0)	0.0830116371842
  (155396, 0)	0.13914395778
  (155993, 0)	0.147208970678
  (156001, 0)	0.0874007032387
  (156656, 0)	0.0820741821209
  (156716, 0)	0.0880868266506
  (156813, 0)	0.0926850587541